# TFL Data Analysis

## Initialisation

In [5]:
import os

import pandas as pd

## Load data

### LSOA lookup

Download the data in csv format from [here](https://data.gov.uk/dataset/9e14de72-df2a-4bbe-b131-1844394e8368/lower-layer-super-output-area-2011-to-ward-2019-lookup-in-england-and-wales) and save it in the `data` directory

Or on the command line:
```
cd data
wget http://geoportal1-ons.opendata.arcgis.com/datasets/15299a7b8e6c498d94a08b687c75b73f_0.csv
```

In [8]:
lsoa_table = pd.read_csv(os.path.join("data", "15299a7b8e6c498d94a08b687c75b73f_0.csv"))
lsoa_table

,FID,LSOA11CD,LSOA11NM,WD19CD,WD19NM,LAD19CD,LAD19NM
0,1,E01012057,Middlesbrough 009E,E05009862,Linthorpe,E06000002,Middlesbrough
1,2,E01012054,Middlesbrough 009B,E05009862,Linthorpe,E06000002,Middlesbrough
2,3,E01012053,Middlesbrough 009A,E05009862,Linthorpe,E06000002,Middlesbrough
3,4,E01033469,Middlesbrough 006E,E05009863,Longlands & Beechwood,E06000002,Middlesbrough
4,5,E01012026,Middlesbrough 006A,E05009863,Longlands & Beechwood,E06000002,Middlesbrough
...,...,...,...,...,...,...,...
34748,34749,W01001322,Merthyr Tydfil 007E,W05000716,Treharris,W06000024,Merthyr Tydfil
34749,34750,W01001320,Merthyr Tydfil 007C,W05000716,Treharris,W06000024,Merthyr Tydfil
34750,34751,W01001321,Merthyr Tydfil 007D,W05000716,Treharris,W06000024,Merthyr Tydfil
34751,34752,W01001324,Merthyr Tydfil 003E,W05000895,Vaynor,W06000024,Merthyr Tydfil
